In [1]:
import torch
import torch.nn as nn
import SETR
import torch.nn.functional as F 
import torchvision 
import numpy as np 
import PIL
from PIL import Image, ImageFilter 
import os 
import random
import glob
import pdb
import math
import random
import time
from torchvision.utils import save_image
import torchvision.transforms.functional as tf 

In [2]:
def get_mean_std(images_list) : 
    pixels = [] 
    for i, filepath in enumerate(images_list) : 
        img = Image.open(filepath)
        try : 
            img = tf.to_tensor(img) 
            pixels.append(img.view(-1))
        except TypeError : 
            print(f'{filepath} is truncated')
        if i % 500 == 0 : 
            print(f'{i}/{len(images_list)}')
        if i == 2000 :
            break # Out of memory..
    pixels = torch.cat(pixels, dim=0)
    return torch.std_mean(pixels, dim=0)

In [3]:
def sync_transform(*images) : 
    w,h = images[0].size # assuming w=512, h<=1024
    assert w == 512 
    if h < 1024 : 
        # pad to 1024
        diff = 1024-h 
        images =  [tf.pad(image, (0,0,diff//2, diff-diff//2)) for image in images]
    w,h = images[0].size
    assert h == 1024
    
    # random horizontal flip
    if random.random() < 0.5 : 
        images = [tf.hflip(image) for image in images]
    
    # random pad
    #if random.random() < 0.5 : 
    #    images = [tf.pad(image, 64) for image in images]
    
    # random rotation
    angle = 0 
    if random.random() < 0.5 : 
        angle = random.randint(-15, 15)
        #images = [tf.rotate(image, angle, resample=PIL.Image.BILINEAR) for image in images]
    
    # random scale 
    scale = 1 
    if random.random() < 0.5 : 
        scale = random.uniform(7/8, 9/8)
    
    images = [tf.affine(image, angle=angle, scale=scale, translate = (0,0), shear=0,
                        resample = PIL.Image.BILINEAR) for image in images]
    
    images = [tf.pad(image, 64) for image in images]
    
    W,H = images[0].size 
    assert H >= h 
    assert W >= w 
    
    h_diff = H-h
    w_diff = W-w 
    
    if random.random() < 0.5 : 
        # Center crop with 50% chance 
        h_start, w_start = h_diff//2, w_diff//2
    else : 
        h_start, w_start = random.randint(0, h_diff), random.randint(0, w_diff)
    
    images = [tf.crop(image, h_start, w_start, h, w) for image in images]
    
    return images

In [4]:
def integer_to_channels(target) : 
    target = (tf.to_tensor(target)*255).int()
    c,h,w = target.shape 
    assert c == 1 
    target = target.squeeze(0).numpy() # (h,w)
    
    output = np.zeros((h,w,3), dtype=np.uint8)
    output[:,:,0] = (target==1).astype(np.uint8)*255
    output[:,:,1] = (target==2).astype(np.uint8)*255
    output[:,:,2] = (target==3).astype(np.uint8)*255
    
    return Image.fromarray(output)

In [5]:
class oct_dataset(object) : 
    def __init__(self, data_path='/../../linux_share/oct_images/images', label_path='/../../linux_share/oct_images/labels', 
                 sync_transform = None, transform=None) : 
        self.data = []
        self.transform=transform
        self.sync_transform = sync_transform
        self.totensor = torchvision.transforms.ToTensor()
        
        filenames = os.listdir(data_path)
        self.data = [(os.path.join(data_path,filename),os.path.join(label_path,filename)) for filename in filenames]
        
            
    def __len__(self) : 
        return len(self.data)
    
    def __getitem__(self, index) : 
        image_path, label_path = self.data[index]
        image_name = os.path.basename(image_path)
        
        
        image = Image.open(image_path)
        label = Image.open(label_path)
        label = integer_to_channels(label)
        
        if self.sync_transform is not None : 
            image, label = self.sync_transform(image, label)
        if self.transform is not None : 
            image = self.transform(image)
            
        image = image.filter(ImageFilter.MedianFilter(size = 5)) 
        
        image = self.totensor(image)
        label = self.totensor(label)
        
        if True : 
            image = tf.normalize(image, 0.1410, 0.0941, inplace=True)
            
        label = label*255 
        
        return image, label,image_name

In [6]:
dataset = oct_dataset(sync_transform=sync_transform)

In [7]:
aux, model = SETR.SETR_Naive_Oct()

In [8]:
print(dataset[0][0].shape)

torch.Size([1, 1024, 512])


C:\Users\UVRLab\.conda\envs\assn1\lib\site-packages\torchvision\transforms\functional.py:1055: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


In [9]:
output = model(dataset[0][0].unsqueeze(0))

C:\Users\UVRLab\.conda\envs\assn1\lib\site-packages\torch\nn\functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


In [10]:
output.shape

torch.Size([1, 3, 1024, 512])